Importing Libraries

In [1]:
import sys
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
print(sys.path)

['', '//anaconda/lib/python27.zip', '//anaconda/lib/python2.7', '//anaconda/lib/python2.7/plat-darwin', '//anaconda/lib/python2.7/plat-mac', '//anaconda/lib/python2.7/plat-mac/lib-scriptpackages', '//anaconda/lib/python2.7/lib-tk', '//anaconda/lib/python2.7/lib-old', '//anaconda/lib/python2.7/lib-dynload', '/Users/senzari/.local/lib/python2.7/site-packages', '/anaconda/lib/python2.7/site-packages/Sphinx-1.3.1-py2.7.egg', '/anaconda/lib/python2.7/site-packages/setuptools-21.2.1-py2.7.egg', '//anaconda/lib/python2.7/site-packages', '//anaconda/lib/python2.7/site-packages/aeosa', '//anaconda/lib/python2.7/site-packages/IPython/extensions', '/Users/senzari/.ipython', '/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py', '/anaconda/envs/stats/lib/python3.5/site-packages', '/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py']


In [2]:
import datetime

import match_stats
import mysql.connector
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import numpy as np
from sklearn import grid_search
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
import matplotlib
from IPython.display import display # Allows the use of display() for DataFrames

import model_libs

Find all matches

In [33]:
cnx = mysql.connector.connect(user='root', password='',
                              host='127.0.0.1',
                              database='mls')
cursor = cnx.cursor(dictionary=True, buffered=True)

match_details = pd.read_sql('SELECT * FROM home_away_coverage_2', cnx)
query = "SELECT id FROM teams"
cursor.execute(query)

round_number = 24
features = {}

upcoming_matches = pd.read_sql("SELECT matches.id as 'match_id', matches.scheduled, matches.home_id, matches.away_id, teams1.full_name AS 'home_team', teams2.full_name AS 'away_team' FROM matches LEFT JOIN teams teams1 ON matches.home_id = teams1.id LEFT JOIN teams teams2 ON matches.away_id = teams2.id WHERE status = 'scheduled'", cnx)
previous_matches = pd.read_sql("SELECT * FROM matches WHERE status = 'closed'", cnx)

print(len(match_details.index))

224


In [48]:
pd.crosstab(match_details['home_score'], 
            match_details.replace({'home_points': {0: 'lose', 1: 'tie', 3: 'win'}})['home_points']).apply(lambda r: r/len(match_details), axis=1)


home_points,lose,tie,win
home_score,,,
0,0.071429,0.066964,0.000000
1,0.062500,0.160714,0.133929
2,0.017857,0.093750,0.183036
3,0.008929,0.008929,0.120536
4,0.000000,0.004464,0.049107
5,0.000000,0.000000,0.017857


In [49]:
pd.crosstab(match_details['away_score'], 
            match_details.replace({'away_points': {0: 'lose', 1: 'tie', 3: 'win'}})['away_points']).apply(lambda r: r/len(match_details), axis=1)

away_points,lose,tie,win
away_score,,,
0,0.281250,0.066964,0.000000
1,0.151786,0.160714,0.031250
2,0.058036,0.093750,0.066964
3,0.013393,0.008929,0.035714
4,0.000000,0.004464,0.022321
7,0.000000,0.000000,0.004464


In [21]:
training_list = []

for team in cursor:
    for i in range(2, round_number):

        print("ROUND {} :: TEAM ID {}".format(i, team["id"]))
        cur_matches = match_details.loc[
            ((match_details['home_id'] == team["id"]) | (match_details['away_id'] == team["id"])) &
            ((match_details['round'] == i))]

        if not cur_matches.empty:
            for c, cur_match in cur_matches.iterrows():
                """ Better Solution for this?  Basically pulling out a Series but the create_match function is expecting a DF
                # have to convert it back to a DF in order to not pull the same entry if there are multiple games in the week """
                temp = pd.DataFrame([])
                df = temp.append(cur_match, ignore_index=True)
                match_result = match_stats.create_match(team["id"], df, match_details, i, False, True)

                if match_result is not None:
                    training_list.append(match_result)                  

columns = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled',
           # Non-Feature Columns
           'games_played', 'is_home', 'avg_points', 'goals_for', 'goals_against', 'avg_goals', 'margin', 'goal_diff',
           'win_percentage', 'sos', 'opp_is_home', 'opp_avg_points', 'opp_avg_goals', 'opp_margin',
           'opp_goal_diff', 'opp_win_percentage', 'opp_opp_record',
           # Extended Feature Columns
           'home_possession', 'away_possession', 'home_attacks', 'away_attacks', 'home_fouls', 'away_fouls',
           'home_yellow_card', 'away_yellow_card', 'home_corner_kicks', 'away_corner_kicks',
           'home_shots_on_target', 'away_shots_on_target', 'home_ball_safe', 'away_ball_safe',
           'home_shots_total', 'away_shots_total',
           'points']  # Target Columns - #'goals', 'opp_goals'

training_data = pd.DataFrame(training_list, columns=columns)
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled', 'goals_for', 'goals_against']
td = model_libs._clone_and_drop(training_data, ignore_cols)
print("Loaded...")

ROUND 2 :: TEAM ID 21
2
 ++++++++++++ 
Team Id : 21 - Name : FC Dallas
Prev Opponent Ids : [25]
FEATURES (Stats from * Previous Matches)
Total Goals : 2
Total Points : 3
Played : 1
Recent Wins : 0 out of 3
Goal Diff : 2
Margin : 2
EXTENDED FEATURES
Home Possession Avg : 46.0
Away Possession Avg : 0.0
Home Attacks Avg : 93.0
Away Attacks Avg : 0.0
Home Fouls Avg : 16.0
Away Fouls Avg : 0.0
Home Yellow Cards Avg : 1.0
Away Yellow Cards Avg : 0.0
Home Corners Avg : 8.0
Away Corners Avg : 0.0
Home Shots On Target Avg : 9.0
Away Shots On Target Avg : 0.0
Home Ball Safe Avg : 87.0
Away Ball Safe Avg : 0.0
Home Goal Attempts Avg : 12.0
Away Goal Attempts Avg : 0.0

TARGETS (RESULTS OF CURRENT MATCH)
Points : 0.0
Goals : 0.0
Opp_Goals : 5.0

Current Opponent ID : 38.0
Team Id : 38.0 - Name : Houston Dynamo
Prev Opponent Ids : [36]
FEATURES (Stats from * Previous Matches)
Total Goals : 3
Total Points : 1
Played : 1
Recent Wins : 0 out of 3
Goal Diff : 0
Margin : 0
EXTENDED FEATURES
Home Possess

In [52]:
display(td.describe())


,games_played,is_home,avg_points,avg_goals,margin,goal_diff,win_percentage,sos,opp_is_home,opp_avg_points,...,away_yellow_card,home_corner_kicks,away_corner_kicks,home_shots_on_target,away_shots_on_target,home_ball_safe,away_ball_safe,home_shots_total,away_shots_total,points
count,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,...,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,11.826087,0.521739,1.260870,1.043478,-0.173913,1.695652,0.671544,1.118511,0.478261,0.913043,...,2.016582,6.026840,4.840821,6.858624,4.223188,89.246910,84.443927,13.265588,7.921946,1.782609
std,6.719390,0.510754,0.540824,0.208514,0.716822,2.898821,0.091929,0.324413,0.510754,0.668312,...,0.699361,0.773174,1.249650,0.832315,1.018249,2.172717,19.335609,1.258853,1.982226,1.346933
min,1.000000,0.000000,1.000000,1.000000,-2.000000,-3.000000,0.500000,0.500000,0.000000,0.000000,...,0.000000,5.125000,0.000000,6.222222,0.000000,85.000000,0.000000,11.888889,0.000000,0.000000
25%,6.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.634146,0.910031,0.000000,1.000000,...,1.572727,5.494048,4.500000,6.400000,4.000000,88.508333,88.777778,12.225000,7.500000,0.500000
50%,11.000000,1.000000,1.000000,1.000000,0.000000,2.000000,0.666667,1.075840,0.000000,1.000000,...,2.166667,5.800000,5.000000,6.428571,4.500000,89.909091,89.833333,12.714286,8.833333,3.000000
75%,17.500000,1.000000,1.000000,1.000000,0.000000,2.000000,0.678571,1.280363,1.000000,1.000000,...,2.500000,6.350000,5.000000,6.784091,4.775000,90.444444,90.500000,14.000000,8.845238,3.000000
max,23.000000,1.000000,3.000000,2.000000,2.000000,7.000000,1.000000,2.000000,1.000000,3.000000,...,3.000000,8.000000,7.000000,9.000000,5.000000,92.500000,93.000000,16.000000,9.800000,3.000000


In [53]:
pd.set_option('display.max_rows', 5000)

In [55]:
def train_classifier(clf, X_train, y_train):
    clf.fit(X_train, y_train)


def predict_labels(clf, features, target):
    y_pred = clf.predict(features)
    print(' ~~~~~~~~~~~~~~~~~~~~~~~ ')
    print(y_pred)
    print(target.values)
    print(' ~~~~~~~~~~~~~~~~~~~~~~~ ')
    return f1_score(target.values, y_pred, average="macro")


def train_predict(clf, X_train, y_train, X_test, y_test):
    train_classifier(clf, X_train, y_train)
    train_f1_score = predict_labels(clf, X_train, y_train)
    test_f1_score = predict_labels(clf, X_test, y_test)

    return train_f1_score, test_f1_score

In [56]:
(y, X) = model_libs._extract_target(td, target_col)
# SVM Model
parameters = {'kernel': ('linear', 'rbf'), 'C': [1, 10]}
svr = SVC()
clf = grid_search.GridSearchCV(svr, parameters)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_f1_score, test_f1_score = train_predict(clf, X_train, y_train, X_test, y_test)

 ~~~~~~~~~~~~~~~~~~~~~~~ 
[ 3.  3.  1.  3.  3.  3.  3.  1.  1.  3.  3.  3.  3.  0.  3.  3.  3.  3.]
[ 3.  3.  1.  3.  3.  3.  3.  1.  1.  3.  3.  3.  1.  0.  3.  1.  0.  3.]
 ~~~~~~~~~~~~~~~~~~~~~~~ 
 ~~~~~~~~~~~~~~~~~~~~~~~ 
[ 3.  3.  3.  3.  3.]
[ 0.  0.  0.  0.  3.]
 ~~~~~~~~~~~~~~~~~~~~~~~ 


//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)
//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)


ValueError: pos_label=1 is not a valid label: array([ 0.,  3.])